import libraries

In [92]:
import tensorflow as tf
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

balancing the dataset to 2000 images in every

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import os
import math
from tqdm import tqdm

data_dir = 'C:/Users/Ayan/OneDrive/Desktop/FER_dataset/train' 
TARGET_COUNT = 7215  # Set to the largest class size or your desired balance count

# Gentle augmentations that keep face structure intact
augmentor = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    if not os.path.isdir(class_path):
        continue

    image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    current_count = len(image_files)
    images_needed = TARGET_COUNT - current_count

    print(f"\n Class '{class_folder}': {current_count} images, needs {images_needed} more.")

    if images_needed <= 0:
        continue

    augmentations_per_image = math.ceil(images_needed / current_count)

    for img_file in tqdm(image_files, desc=f"Augmenting {class_folder}"):
        img_path = os.path.join(class_path, img_file)
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        aug_count = 0
        for batch in augmentor.flow(x, batch_size=1):
            aug_img = array_to_img(batch[0])
            save_name = f"{os.path.splitext(img_file)[0]}_aug{aug_count}.jpg"
            aug_img.save(os.path.join(class_path, save_name))
            aug_count += 1
            if aug_count >= augmentations_per_image:
                break

        if len(os.listdir(class_path)) >= TARGET_COUNT:
            break

print("\nDataset is face-balance ready")


 Class 'angry': 7215 images, needs 0 more.

 Class 'disgust': 7215 images, needs 0 more.

 Class 'fear': 7215 images, needs 0 more.

 Class 'happy': 7215 images, needs 0 more.

 Class 'neutral': 7215 images, needs 0 more.

 Class 'sad': 7215 images, needs 0 more.

 Class 'surprise': 7215 images, needs 0 more.

Dataset is face-balance ready


Part:-1 preprocessing the tarin and validation data

In [113]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create the data generator - no color_mode here
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Use color_mode in flow_from_directory instead
train_data = datagen.flow_from_directory(
    'C:/Users/Ayan/OneDrive/Desktop/FER_dataset/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',  # This is correct - color_mode goes here
    subset='training'
)

val_data = datagen.flow_from_directory(
    'C:/Users/Ayan/OneDrive/Desktop/FER_dataset/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',  # This is correct - color_mode goes here
    subset='validation'
)

Found 40404 images belonging to 7 classes.
Found 10101 images belonging to 7 classes.


Part-2:_ buildinfg the cnn

In [118]:
# Create model with correct input shape for grayscale
cnn = tf.keras.models.Sequential([
    # First layer with input_shape=(128, 128, 1) for grayscale
    Conv2D(32, kernel_size=3, activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D(pool_size=2, strides=2),
    
    # Remaining layers
    Conv2D(64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2, strides=2),
    
    Conv2D(128, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2, strides=2),
    
    Flatten(),
    
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 14, 14, 128)      

Part-3:- Train the model

In [ ]:
model = cnn.fit(
    train_data,
    epochs=15,
    validation_data=val_data
)

Epoch 1/15
1263/1263 [==============================] - 3971s 3s/step - loss: 1.7468 - accuracy: 0.3081 - val_loss: 1.5711 - val_accuracy: 0.3930
Epoch 2/15
1263/1263 [==============================] - 2097s 2s/step - loss: 1.4712 - accuracy: 0.4427 - val_loss: 1.4681 - val_accuracy: 0.4433
Epoch 3/15
1263/1263 [==============================] - 2362s 2s/step - loss: 1.3381 - accuracy: 0.4954 - val_loss: 1.4065 - val_accuracy: 0.4716
Epoch 4/15
 399/1263 [========>.....................] - ETA: 21:26 - loss: 1.2421 - accuracy: 0.5290